<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Face_Similarity_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Reverse Image Search System for Defensive Forensics**

In [1]:
!pip install -q torch transformers langchain langgraph numpy pillow requests vllm langchain_community groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import asyncio
import os
import torch
import numpy as np
import requests
from io import BytesIO
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import matplotlib.pyplot as plt
import networkx as nx
import aiohttp
import time
from typing import Dict, List, Any, Tuple, Optional, Union
import json

# For embeddings and multimodal processing
from transformers import CLIPProcessor, CLIPModel, AutoProcessor, AutoModel
import torch.nn.functional as F

# For LLM orchestration
from groq import Groq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages import SystemMessage, HumanMessage

# For graph-based agent orchestration
from langgraph.graph import StateGraph, END

In [4]:
import getpass

# Securely get API keys from user input
GROQ_API_KEY = getpass.getpass("Enter your GROQ API Key: ")
GOOGLE_CSE_ID = getpass.getpass("Enter your Google CSE ID: ")
GOOGLE_API_KEY = getpass.getpass("Enter your Google API Key: ")

Enter your GROQ API Key: ··········
Enter your Google CSE ID: ··········
Enter your Google API Key: ··········


In [5]:
# Initialize global models
print("Loading models...")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Load BLIP-2 for better image understanding
blip_processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
blip_model = AutoModel.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

# Load DINOv2 for more robust feature extraction
dinov2_model = AutoModel.from_pretrained("facebook/dinov2-base")
dinov2_processor = AutoProcessor.from_pretrained("facebook/dinov2-base")

# Set device (use GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = clip_model.to(device)
blip_model = blip_model.to(device)
dinov2_model = dinov2_model.to(device)

print(f"Models loaded on {device}")

llm = Groq(
    api_key=GROQ_API_KEY,
    model_name="mixtral-8x7b-32768",
    temperature=0.1
)

Loading models...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Models loaded on cuda


TypeError: Groq.__init__() got an unexpected keyword argument 'model_name'